# Analyse data

In [36]:
import pandas as pd
import json
from pathlib import Path
import utils
import datetime

In [37]:
df_votum_raw = pd.read_csv(Path('../export/votum.csv'))
df_votum_raw['sitzung_date'] = pd.to_datetime(df_votum_raw['sitzung_date'])

with open(Path('../export/mitglieder.json'), encoding='utf-8') as f:
    kantonsrat = json.load(f)

# Typecast
utils.kantonsrat_to_datetime(kantonsrat)

## How many times did m/w talk?

In [38]:
df_votum = df_votum_raw.copy()

# Remove all Statements from President, 1. & 2. Vizepresidents
df_votum = df_votum[~df_votum.funktion.isin(['Präsidium', '2. Vizepräsidium', '1. Vizepräsidium'])]

# Calculate Amtsjahr (Amtsjahr 2020 = 1.5.2020 - 30.4.2021)
df_votum['year'] = df_votum['sitzung_date'].dt.year
df_votum['month'] = df_votum['sitzung_date'].dt.month
df_votum['session'] = df_votum.apply(lambda row: row['year'] if row['month'] > 5 else row['year'] - 1, axis=1)

In [42]:
# Calculate m/w percentage of Kantonsrat
year = 2020
dt = datetime.datetime(year, 11, 11)
#len(list(filter(lambda e: (e['start'] <= dt) and (e['end'] >= dt), x))) > 0
kant2020 = filter(lambda x: len(list(filter(lambda e: (e['start'] <= dt) and (e['end'] >= dt), x['einsitz']))) > 0, kantonsrat)

records = []
for r in list(kant2020):
    records.append({'name': r['name'], 'vorname': r['vorname'], 'start': r['einsitz'][0]['start'], 'end': r['einsitz'][0]['end']})

df = pd.DataFrame(records)

In [43]:
df.to_clipboard()

In [44]:
len(df)

180

In [25]:
pd.pivot_table(df_votum, index='session', columns='geschlecht', values='name', aggfunc='count')

geschlecht,m,w
session,,
2012,848,324
2013,2933,1089
2014,2382,794
2015,2265,911
2016,2214,905
2017,2139,903
2018,2460,986
2019,1967,1001
2020,2332,1189


In [23]:
df_votum.funktion.unique()

array(['Mitglied', nan], dtype=object)

In [20]:
df_votum.head()

,name,vorname,text,f,sitzung_name,sitzung_date,sitzung_gremium,partei,geschlecht,jahrgang,funktion,year,month,session
0,Egg,Bernhard,"Zuerst wünsche ich Ihnen allen, auch wenn scho...",2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,SP,m,1959.0,Präsidium,2013,1,2012
1,Egg,Bernhard,Dann hat mir Toni Schaller (Altkantonsrat Anto...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,SP,m,1959.0,Präsidium,2013,1,2012
2,Egg,Bernhard,Wir dürfen für den aus dem Kantonsrat zurückge...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,SP,m,1959.0,Präsidium,2013,1,2012
3,Egg,Bernhard,"Ich bitte, den Gewählten eintreten zu lassen. ...",2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,SP,m,1959.0,Präsidium,2013,1,2012
4,Egg,Bernhard,«Ich gelobe es.»,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332,88. KR Sitzung,2013-01-14,KR,SP,m,1959.0,Präsidium,2013,1,2012
